# LSTM으로 텍스트 생성하기

## 글자 수준의 LSTM 텍스트 생성 모델 구현

이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.

https://s3.amazonaws.com/text-datasets/nietzsche.txt

In [36]:
# 과연 이 모델은 신이 죽었는지 아닌지에 대해 말할 것인가?

from tensorflow.keras import utils
import numpy as np, tensorflow as tf

path = utils.get_file('nietzche.txt',
                      origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower() # 가서 보면 알겠지만 중간에 대문자 많이 튀어나옴

print('말뭉치 크기:',len(text))

말뭉치 크기: 600893


In [37]:
# 내용 맛보기. 개행 문자와 특문을 처리해야 할 것 같지만 일단 이대로 진행.(문장 부호도 하나의 특징이니까)
text[:300]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, ha'

In [38]:
# 60개 글자로 된 시퀀스를 추출. 한 문장 안에 알파벳 60개가 들어 있게끔.
maxlen = 60

# 샘플링은 3글자씩 건너뛰면서 새로운 시퀀스를 샘플링.
# n_gram이 3인 것과 같은 뜻일까?
step = 3

sentence = [] # 시퀀스는 여기 리스트에
target = []   # 해당 시퀀스 다음 글자는 이 리스트에

for i in range(0, len(text)-maxlen, step): # 0에서 600,833까지. 마지막 시퀀스가 60글자인 걸 감안해야지.
  sentence.append(text[i:i+maxlen])
  target.append(text[i+maxlen])

print('시퀀스 총 개수:', len(sentence))
print(sentence[:2])

시퀀스 총 개수: 200278
['preface\n\n\nsupposing that truth is a woman--what then? is the', 'face\n\n\nsupposing that truth is a woman--what then? is there ']


In [39]:
import warnings; warnings.filterwarnings('ignore')

# 말뭉치에서 고유한 글자를 담은 리스트. 알파벳 26자와 특문 등이 들어갈 것이다.
char = sorted(list(set(text)))
print('고유한 글자 수:',len(char))

# 고유 글자와 해당 글자의 인덱스를 딕셔너리화
char_dict = dict((character, char.index(character)) for character in char)

# 원-핫 인코딩
x = np.zeros((len(sentence), maxlen, len(char)), dtype=np.bool) # boolean 타입은 true or false의 2가지 값만 가진다 했다.
y = np.zeros((len(sentence), len(char)), dtype=np.bool)

for i, sen in enumerate(sentence):
  for t, ch in enumerate(sen):
    x[i, t, char_dict[ch]] = 1
  y[i, char_dict[target[i]]] = 1

고유한 글자 수: 57


In [40]:
# 네트워크 구성
from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(char))))
model.add(layers.Dense(len(char),activation='softmax'))

# 위의 모델이 무슨 말이냐 하면, 시퀀스와 그 이후 오는 글자를 학습한 후에
# 저 고유 글자들 중에서 어떤 글자가 다음에 올지 그 확률이 제일 높은 걸
# 다음 글자로 선정한다는 것임.
# 이때 '소프트맥스 온도'라는 개념이 사용되는데, 엔트로피의 높낮이를 바꾸어 무작위성에 차이를 둘 수 있음
# 뻔한 글자가 나올 수도, 창의적인 글자가 나올 수도 있게.

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01),
              loss='categorical_crossentropy')
# 여기선 뭐 분류하고 그런 거 아니라서 정확도 측정은 안 해도 된다. 정답이 없으니까.

## 언어 모델 훈련과 샘플링

훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다):


밑이 자연상수 e인 지수함수(e^x)의 그래프<br>
https://wooono.tistory.com/214

Python Numpy.log()-로그 <br>
https://www.delftstack.com/ko/api/numpy/python-numpy-log/


In [41]:
# 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
def sample(pred, temperature=1.0):
  pred = np.asarray(pred).astype('float64')
  pred = np.log(pred)/temperature
  exp_pred = np.exp(pred)
  pred = exp_pred / np.sum(exp_pred)
  prob = np.random.multinomial(1, pred, 1)
  return np.argmax(prob)

마지막으로 다음 반복문은 반복적으로 훈련하고 텍스트를 생성합니다. 에포크마다 학습이 끝난 후 여러가지 온도를 사용해 텍스트를 생성합니다. 이렇게 하면 모델이 수렴하면서 생성된 텍스트가 어떻게 진화하는지 볼 수 있습니다. 온도가 샘플링 전략에 미치는 영향도 보여 줍니다.

In [42]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print('학습할 에포크 횟수:', epoch)
    # 매 1번씩 새로이 학습. 이를 총 60번
    model.fit(x, y, batch_size=128, epochs=1)

    # 바탕이 될 시드 텍스트는 무작위로 선택
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 소프트맥스 온도가 낮을수록 뻔하지만 논리적이고, 높을수록 참신하고 못 알아들을 말이 나온다.
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트로부터 400개의 글자를 덧붙일 예정
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩
            sampled = np.zeros((1, maxlen, len(char)))
            for t, character in enumerate(generated_text):
                sampled[0, t, char_dict[character]] = 1.

            # 다음 글자를 샘플링해 가면서 해당 온도에 맞는 문장을 붙여 출력
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = char[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

학습할 에포크 횟수: 1
1565/1565 [==============================] - 229s 145ms/step - loss: 1.9730
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the same and the sensister the spirit the sengution with the sensistance the senguth and the sensistance of the sensistance the sengust the seneres and the sengute the sengute the sensister and the sensister and the spirit in the sensister the sengured to the sengute and the sensust the sengute the passions and the senguth and the spirits the sengute and concestion and striggt in the senguth the
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for know, the concernte it everything the conscions of the most the feal to the longer and all the moral man as all the domand the sensicts of the spirits of the conscious to in restrotions of the religious for with it stander as the prich the belief the this sought and confurse the as